In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 환경설정

In [3]:
# 파이토치 버전 확인
import torch

torch.__version__

'2.0.1+cu118'

In [6]:
# 파이토치 버전 다운그레이드
!pip install torch --downgrade version==1.9.0


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --downgrade


In [7]:
  # 파이썬 버전 변경
  !wget https://www.python.org/ftp/python/3.6.9/Python-3.6.9.tgz
  !tar xvfz Python-3.6.9.tgz
  !Python-3.6.9/configure
  !make
  !sudo make install

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Compiling '/usr/local/lib/python3.6/encodings/iso8859_13.py'...
Compiling '/usr/local/lib/python3.6/encodings/iso8859_14.py'...
Compiling '/usr/local/lib/python3.6/encodings/iso8859_15.py'...
Compiling '/usr/local/lib/python3.6/encodings/iso8859_16.py'...
Compiling '/usr/local/lib/python3.6/encodings/iso8859_2.py'...
Compiling '/usr/local/lib/python3.6/encodings/iso8859_3.py'...
Compiling '/usr/local/lib/python3.6/encodings/iso8859_4.py'...
Compiling '/usr/local/lib/python3.6/encodings/iso8859_5.py'...
Compiling '/usr/local/lib/python3.6/encodings/iso8859_6.py'...
Compiling '/usr/local/lib/python3.6/encodings/iso8859_7.py'...
Compiling '/usr/local/lib/python3.6/encodings/iso8859_8.py'...
Compiling '/usr/local/lib/python3.6/encodings/iso8859_9.py'...
Compiling '/usr/local/lib/python3.6/encodings/johab.py'...
Compiling '/usr/local/lib/python3.6/encodings/koi8_r.py'...
Compiling '/usr/local/lib/python3.6/encodings/koi8_t.py'...
Compiling '/usr/local/lib

In [8]:
# 경로 설정
cd "/content/drive/MyDrive/Colab Notebooks/Kjw/LTSF"

SyntaxError: ignored

#### 깃 클론

In [ ]:
!git clone https://github.com/cure-lab/LTSF-Linear.git

In [ ]:
# 코랩에 파일 불러오기
from google.colab import files
src=list(files.upload().values())[0]

## electrocity

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from datetime import datetime, timedelta
import pandas as pd
import math
import numpy as np
import random
from tqdm import trange

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

from math import sqrt
from pandas import read_csv, DataFrame
from scipy import stats

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt


def prep_data(data, covariates, data_start, train = True):
    """Divide the training sequence into windows"""
    time_len = data.shape[0]
    input_size = window_size-stride_size
    windows_per_series = np.full((num_series), (time_len-input_size) // stride_size)
    if train: windows_per_series -= (data_start+stride_size-1) // stride_size
    total_windows = np.sum(windows_per_series)
    x_input = np.zeros((total_windows, window_size, 1 + num_covariates + 1), dtype='float32')
    label = np.zeros((total_windows, window_size), dtype='float32')
    v_input = np.zeros((total_windows, 2), dtype='float32')
    count = 0
    if not train:
        covariates = covariates[-time_len:]
    for series in trange(num_series):
        cov_age = stats.zscore(np.arange(total_time-data_start[series])) # shape:(series_len,)
        if train:
            covariates[data_start[series]:time_len, 0] = cov_age[:time_len-data_start[series]]
        else:
            covariates[:, 0] = cov_age[-time_len:]
        for i in range(windows_per_series[series]):
            if train:
                window_start = stride_size*i+data_start[series]
            else:
                window_start = stride_size*i
            window_end = window_start+window_size
            '''
            print("x: ", x_input[count, 1:, 0].shape)
            print("window start: ", window_start)
            print("window end: ", window_end)
            print("data: ", data.shape)
            print("d: ", data[window_start:window_end-1, series].shape)
            '''
            x_input[count, 1:, 0] = data[window_start:window_end-1, series]
            x_input[count, :, 1:1+num_covariates] = covariates[window_start:window_end, :]
            x_input[count, :, -1] = series
            label[count, :] = data[window_start:window_end, series]
            nonzero_sum = (x_input[count, 1:input_size, 0]!=0).sum()
            if nonzero_sum == 0:
                v_input[count, 0] = 0
            else:
                v_input[count, 0] = np.true_divide(x_input[count, 1:input_size, 0].sum(),nonzero_sum)+1
                x_input[count, :, 0] = x_input[count, :, 0]/v_input[count, 0]
                if train:
                    label[count, :] = label[count, :]/v_input[count, 0]
            count += 1
    prefix = os.path.join(save_path, 'train_' if train else 'test_')
    np.save(prefix+'data_'+save_name, x_input)
    np.save(prefix+'v_'+save_name, v_input)
    np.save(prefix+'label_'+save_name, label)

def gen_covariates(times, num_covariates):
    """Get covariates"""
    covariates = np.zeros((times.shape[0], num_covariates))
    for i, input_time in enumerate(times):
        covariates[i, 1] = input_time.weekday()
        covariates[i, 2] = input_time.hour
        covariates[i, 3] = input_time.month
    for i in range(1,num_covariates):
        covariates[:,i] = stats.zscore(covariates[:,i])
    return covariates[:, :num_covariates]

def visualize(data, week_start):
    x = np.arange(window_size)
    f = plt.figure()
    plt.plot(x, data[week_start:week_start+window_size], color='b')
    f.savefig("visual.png")
    plt.close()



In [ ]:
if __name__ == '__main__':

    global save_path
    csv_path = '/content/drive/MyDrive/Colab Notebooks/Kjw/LTSF/electricity.csv'
    save_name = 'elect'
    window_size = 192
    stride_size = 24
    num_covariates = 4
    train_start = '2011-01-01 00:00:00'
    train_end = '2014-08-31 23:00:00'
    test_start = '2014-08-25 00:00:00' #need additional 7 days as given info
    test_end = '2014-09-07 23:00:00'
    pred_days = 7
    given_days = 7

    save_path = os.path.join('data', save_name)

    data_frame = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kjw/LTSF/electricity.csv')
    data_frame['date']=pd.to_datetime(data_frame['date'], errors='raise', format='%Y-%m-%d %H:%M')
    data_frame = data_frame.set_index('date')
    data_frame = data_frame.resample('1H',label = 'left',closed = 'right').sum()[train_start:test_end]
    data_frame.fillna(0, inplace=True)
    covariates = gen_covariates(data_frame[train_start:test_end].index, num_covariates)
    train_data = data_frame[train_start:train_end].values # shape: [seq_length, user_num]
    test_data = data_frame[test_start:test_end].values
    if train_data.size != 0:
        data_start = (train_data!=0).argmax(axis=0) #find first nonzero value in each time series
        # if 문이 안되면 이걸로 사용 data_start = (train_data!=0).argmax(axis=0)
        total_time = data_frame.shape[0] #32304
        num_series = data_frame.shape[1] #370
        prep_data(train_data, covariates, data_start)
        prep_data(test_data, covariates, data_start, train=False)

# 데이터 불러오기

In [11]:
# 데이터 분석
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

import warnings
warnings.filterwarnings(action='ignore')

from datetime import datetime
import calendar

plt.rc('font', family='NanumGothic')
plt.style.use('ggplot')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# sklearn
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import KFold

# 인코더
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelEncoder

# 스케일러
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report
# from sklearn.metrics import silhouette_samples, silhouette_score
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report
# from sklearn.metrics import r2_score

# from sklearn.compose import make_column_transformer

# from sklearn.pipeline import Pipeline, make_pipeline

# from sklearn.cluster import KMeans

# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier

# 캣부스트
# from catboost import CatBoostClassifier
# from catboost import CatBoostRegressor

# 옵튜나
# import optuna
# from optuna import Trial
# from optuna.samplers import TPESampler

# plt.rcParams['font.family'] = 'NanumBarunGothic' # 나눔바른고딕 적용하기

# 자연어 처리
import re

# 파이캐럿
# from pycaret.regression import *

from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder
import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [12]:
brand=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/brand_keyword_cnt.csv')
product=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/product_info.csv')
sales=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/sales.csv')
train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/train.csv')
submission=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/sample_submission.csv')

# LTSF Linear

In [14]:
Train=train.drop(['ID','제품','대분류','중분류','소분류','브랜드'],axis=1)
Train.shape

(15890, 459)

In [ ]:
class LTSF_Linear(torch.nn.Module):
    def __init__(self, window_size, forcast_size, individual, feature_size):
        super(LTSF_Linear, self).__init__() # super() : 이 함수는 super class 즉, 부모클래스의 임시적인 객체를 반환하여 부모클래스의 메소드를 사용할 수 있게 하는 것
        self.window_size = window_size #
        self.forcast_size = forcast_size
        self.individual = individual
        self.channels = feature_size
        if self.individual:
            self.Linear = torch.nn.ModuleList()
            for i in range(self.channels):
                self.Linear.append(torch.nn.Linear(self.window_size, self.forcast_size))
        else:
            self.Linear = torch.nn.Linear(self.window_size, self.forcast_size)

    def forward(self, x):
        if self.individual:
            output = torch.zeros([x.size(0),self.pred_len,x.size(2)],dtype=x.dtype).to(x.device)
            for i in range(self.channels):
                output[:,:,i] = self.Linear[i](x[:,:,i])
            x = output
        else:
            x = self.Linear(x.permute(0,2,1)).permute(0,2,1)
        return x

In [ ]:
# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = train_data.columns[4:] # 날짜 데이터의 컬럼들

# 각 column의 min 및 max 계산 -> 데이터에 minmaxscaling 하기 위해서 값 찾기
min_values = train_data[numeric_cols].min(axis=1) # 데이터프레임에서 최솟값 -> 각 행마다 최소값을 출력함(제품 하나 당 행 하나)
max_values = train_data[numeric_cols].max(axis=1) # 데이터프레임에서 최댓값

# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체 -> 최대-최소값이 분모에 들어가기 때문에 0이 되면 안됨.
ranges = max_values - min_values
ranges[ranges == 0] = 1

# min-max scaling 수행
train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0) # subtract는 np에서 뺄셈 연산. axis=0이므로 열별로 뺄셈 수행.
# 위의 식은 min max scaler 공식임. (x-min)/max-min

# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()

In [ ]:
CFG={
     'TRAIN_WINDOW_SIZE':400,
     'PREDICT_SIZE':21,
     'seed':0
     }

def make_train_data(train, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']): # 딕셔너리
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''

    train_data=Train.copy()

    train_df=train_data[:300]
    test_df=train[301:438]


def minmaxscaling(train_df, test_df):
    train_df = train_data.copy()
    test_df_ = test_df.copy()

    numeric_cols = train_data.columns

    min_values = train_df[numeric_cols].min(axis=1) # 데이터프레임에서 최솟값 -> 각 행마다 최소값을 출력함(제품 하나 당 행 하나)
    max_values = train_df[numeric_cols].max(axis=1) # 데이터프레임에서 최댓값

    ranges = max_values - min_values
    ranges[ranges == 0] = 1

    train_df[numeric_cols] = (train_df[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)

    scale_min_dict = min_values.to_dict()
    scale_max_dict = max_values.to_dict()

    return scale_min_dict. scale_max_dict

def time_slide_df(df, window_size, forcast_size, date, target):
    train_data_ = train_data.copy()
    data_list = []
    dap_list = []
    date_list = []
    for idx in range(0, train_data_.shape[1]-window_size-forcast_size+1):
        x = train_data_.loc[:, target].values.reshape(window_size, 1)
        y = train_data_.loc[idx+window_size:idx+window_size+forcast_size-1, target].values
        date_ = train_data_.loc[idx+window_size:idx+window_size+forcast_size-1, date].values
        data_list.append(x)
        dap_list.append(y)
        date_list.append(date_)
    return np.array(data_list, dtype='float32'), np.array(dap_list, dtype='float32'), np.array(date_list)

class Data(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

### Univariable ###
### 데이터 셋 생성 ###
window_size = 72
forcast_size= 24 # 24시간이라서
batch_size = 32
targets = '전력사용량(kWh)'
date = 'date'

train_df, test_df = minmaxscaling(train_df, test_df)
train_x, train_y, train_date = time_slide_df(train_df_fe, window_size, forcast_size, date, targets)
test_x, test_y, test_date = time_slide_df(test_df_fe, window_size, forcast_size, date, targets)

train_ds = Data(train_x[:1000], train_y[:1000])
valid_ds = Data(train_x[1000:], train_y[1000:])
test_ds = Data(test_x, test_y)

train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle=True,)
valid_dl = DataLoader(valid_ds, batch_size = train_x[1000:].shape[0], shuffle=False)
test_dl  = DataLoader(test_ds,  batch_size = test_x.shape[0], shuffle=False)

# ### Univariable ###
# ### 데이터 셋 생성 ###
# window_size = 72
# forcast_size= 24 # 24시간이라서
# batch_size = 32
# targets = '전력사용량(kWh)'
# date = 'date_time'

# train_df_fe, test_df_fe, mean_, std_ = standardization(train_df, test_df, 'date_time', targets)
# train_x, train_y, train_date = time_slide_df(train_df_fe, window_size, forcast_size, date, targets)
# test_x, test_y, test_date = time_slide_df(test_df_fe, window_size, forcast_size, date, targets)

# train_ds = Data(train_x[:1000], train_y[:1000])
# valid_ds = Data(train_x[1000:], train_y[1000:])
# test_ds = Data(test_x, test_y)

# train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle=True,)
# valid_dl = DataLoader(valid_ds, batch_size = train_x[1000:].shape[0], shuffle=False)
# test_dl  = DataLoader(test_ds,  batch_size = test_x.shape[0], shuffle=False)


### 모델 학습 ###
train_loss_list = []
valid_loss_list = []
test_loss_list = []
epoch = 50
lr = 0.001
DLinear_model = LTSF_DLinear(
                            window_size=window_size,
                            forcast_size=forcast_size,
                            kernel_size=25,
                            individual=False,
                            feature_size=1,
                            )
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(DLinear_model.parameters(), lr=lr)
max_loss = 999999999

for epoch in tqdm(range(1, epoch+1)):
    loss_list = []
    DLinear_model.train()
    for batch_idx, (data, target) in enumerate(train_dl):
        optimizer.zero_grad()
        output = DLinear_model(data)
        loss = criterion(output, target.unsqueeze(-1))
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_loss_list.append(np.mean(loss_list))

    DLinear_model.eval()
    with torch.no_grad():
        for data, target in valid_dl:
            output = DLinear_model(data)
            valid_loss = criterion(output, target.unsqueeze(-1))
            valid_loss_list.append(valid_loss)

        for data, target in test_dl:
            output = DLinear_model(data)
            test_loss = criterion(output, target.unsqueeze(-1))
            test_loss_list.append(test_loss)

    if valid_loss < max_loss:
        torch.save(DLinear_model, 'DLinear_model.pth')
        max_loss = valid_loss
        print("valid_loss={:.3f}, test_los{:.3f}, Model Save".format(valid_loss, test_loss))
        dlinear_best_epoch = epoch
        dlinear_best_train_loss = np.mean(loss_list)
        dlinear_best_valid_loss = np.mean(valid_loss.item())
        dlinear_best_test_loss = np.mean(test_loss.item())

    print("epoch = {}, train_loss : {:.3f}, valid_loss : {:.3f}, test_loss : {:.3f}".format(epoch, np.mean(loss_list), valid_loss, test_loss))

# 개념들

- 클래스의 사용 목적 : 변수와 함수를 묶어서 하나의 새로운 객체(타입)으로 만드는 것
- '클래스를 정의한다.' 의 의미 : 새로운 데이터 타입을 정의한 것. 이를 실제로 사용하려면 인스턴스를 생성해야함.
- 인스턴스 : 붕어빵 틀(클래스)에 반죽을 넣어서 만들어진 붕어빵

In [ ]:
# 정의된 클래스를 이용해서 인스턴스 생성하기 : 클래스 이름 뒤에 ()를 넣으면 된다.
>>> card1 = BusinessCard()
>>> card1 # 인스턴스
<__main__.BusinessCard object at 0x0302ABB0>
# BusinessCard라는 클래스의 인스턴스가 메모리의 한 위치에 생성되고, card1이라는 변수가 이를 바인딩하게 된다.

In [ ]:
# 클래스에 메서드 추가하기
>>> class BusinessCard:
        def set_info(self, name, email, addr):
                self.name = name
                self.email = email
                self.addr = addr

# 모델링

In [ ]:
class moving_avg(torch.nn.Module):
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = torch.nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x

class series_decomp(torch.nn.Module):
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        residual = x - moving_mean
        return moving_mean, residual

 class LTSF_DLinear(torch.nn.Module):
    def __init__(self, window_size, forcast_size, kernel_size, individual, feature_size):
        super(LTSF_DLinear, self).__init__()
        self.window_size = window_size
        self.forcast_size = forcast_size
        self.decompsition = series_decomp(kernel_size)
        self.individual = individual
        self.channels = feature_size
        if self.individual:
            self.Linear_Seasonal = torch.nn.ModuleList()
            self.Linear_Trend = torch.nn.ModuleList()
            for i in range(self.channels):
                self.Linear_Trend.append(torch.nn.Linear(self.window_size, self.forcast_size))
                self.Linear_Trend[i].weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
                self.Linear_Seasonal.append(torch.nn.Linear(self.window_size, self.forcast_size))
                self.Linear_Seasonal[i].weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
        else:
            self.Linear_Trend = torch.nn.Linear(self.window_size, self.forcast_size)
            self.Linear_Trend.weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
            self.Linear_Seasonal = torch.nn.Linear(self.window_size,  self.forcast_size)
            self.Linear_Seasonal.weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))

    def forward(self, x):
        trend_init, seasonal_init = self.decompsition(x)
        trend_init, seasonal_init = trend_init.permute(0,2,1), seasonal_init.permute(0,2,1)
        if self.individual:
            trend_output = torch.zeros([trend_init.size(0), trend_init.size(1), self.forcast_size], dtype=trend_init.dtype).to(trend_init.device)
            seasonal_output = torch.zeros([seasonal_init.size(0), seasonal_init.size(1), self.forcast_size], dtype=seasonal_init.dtype).to(seasonal_init.device)
            for idx in range(self.channels):
                trend_output[:, idx, :] = self.Linear_Trend[idx](trend_init[:, idx, :])
                seasonal_output[:, idx, :] = self.Linear_Seasonal[idx](seasonal_init[:, idx, :])
        else:
            trend_output = self.Linear_Trend(trend_init)
            seasonal_output = self.Linear_Seasonal(seasonal_init)
        x = seasonal_output + trend_output
        return x.permute(0,2,1)

IndentationError: ignored

In [ ]:
def standardization(train_df, test_df, not_col, target):
    train_df_ = train_df.copy()
    test_df_ = test_df.copy()
    col =  [col for col in list(train_df.columns) if col not in [not_col]]
    mean_list = []
    std_list = []
    for x in col:
        mean, std = train_df_.agg(["mean", "std"]).loc[:,x]
        mean_list.append(mean)
        std_list.append(std)
        train_df_.loc[:, x] = (train_df_[x] - mean) / std
        test_df_.loc[:, x] = (test_df_[x] - mean) / std
    return train_df_, test_df_, mean_list[col.index(target)], std_list[col.index(target)]

def time_slide_df(df, window_size, forcast_size, date, target):
    df_ = df.copy()
    data_list = []
    dap_list = []
    date_list = []
    for idx in range(0, df_.shape[0]-window_size-forcast_size+1):
        x = df_.loc[idx:idx+window_size-1, target].values.reshape(window_size, 1)
        y = df_.loc[idx+window_size:idx+window_size+forcast_size-1, target].values
        date_ = df_.loc[idx+window_size:idx+window_size+forcast_size-1, date].values
        data_list.append(x)
        dap_list.append(y)
        date_list.append(date_)
    return np.array(data_list, dtype='float32'), np.array(dap_list, dtype='float32'), np.array(date_list)

class Data(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

### Univariable ###
### 데이터 셋 생성 ###
window_size = 72
forcast_size= 24
batch_size = 32
targets = '전력사용량(kWh)'
date = 'date_time'

train_df_fe, test_df_fe, mean_, std_ = standardization(train_df, test_df, 'date_time', targets)
train_x, train_y, train_date = time_slide_df(train_df_fe, window_size, forcast_size, date, targets)
test_x, test_y, test_date = time_slide_df(test_df_fe, window_size, forcast_size, date, targets)

train_ds = Data(train_x[:1000], train_y[:1000])
valid_ds = Data(train_x[1000:], train_y[1000:])
test_ds = Data(test_x, test_y)

train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle=True,)
valid_dl = DataLoader(valid_ds, batch_size = train_x[1000:].shape[0], shuffle=False)
test_dl  = DataLoader(test_ds,  batch_size = test_x.shape[0], shuffle=False)


### 모델 학습 ###
train_loss_list = []
valid_loss_list = []
test_loss_list = []
epoch = 50
lr = 0.001
DLinear_model = LTSF_DLinear(
                            window_size=window_size,
                            forcast_size=forcast_size,
                            kernel_size=25,
                            individual=False,
                            feature_size=1,
                            )
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(DLinear_model.parameters(), lr=lr)
max_loss = 999999999

for epoch in tqdm(range(1, epoch+1)):
    loss_list = []
    DLinear_model.train()
    for batch_idx, (data, target) in enumerate(train_dl):
        optimizer.zero_grad()
        output = DLinear_model(data)
        loss = criterion(output, target.unsqueeze(-1))
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_loss_list.append(np.mean(loss_list))

    DLinear_model.eval()
    with torch.no_grad():
        for data, target in valid_dl:
            output = DLinear_model(data)
            valid_loss = criterion(output, target.unsqueeze(-1))
            valid_loss_list.append(valid_loss)

        for data, target in test_dl:
            output = DLinear_model(data)
            test_loss = criterion(output, target.unsqueeze(-1))
            test_loss_list.append(test_loss)

    if valid_loss < max_loss:
        torch.save(DLinear_model, 'DLinear_model.pth')
        max_loss = valid_loss
        print("valid_loss={:.3f}, test_los{:.3f}, Model Save".format(valid_loss, test_loss))
        dlinear_best_epoch = epoch
        dlinear_best_train_loss = np.mean(loss_list)
        dlinear_best_valid_loss = np.mean(valid_loss.item())
        dlinear_best_test_loss = np.mean(test_loss.item())

    print("epoch = {}, train_loss : {:.3f}, valid_loss : {:.3f}, test_loss : {:.3f}".format(epoch, np.mean(loss_list), valid_loss, test_loss))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-100-c2765f1d31f6>", line 29, in <cell line: 29>
    class Data(Dataset):
NameError: name 'Dataset' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core